# Generalized use of a toolbox for network simulations

In [1]:
import numpy as np

from scenario import Scenario 
from network import TrafficNetwork 
from demand import Demand, TrafficDemand 
from carfollow import K_X, W_I, U_I

from bokeh.io import output_notebook
from bokeh.plotting import show

output_notebook()

Loading BokehJS ...

In [2]:
C = U_I * W_I * K_X / (U_I + W_I) * 3600  # veh /h
T_SIM = 720 # simulation time 
MPR = 0.5

np.random.seed(42)  # Reproducibility MPR

## Traffic network 

In this case a simple traffic network is created by providing two single links one with 2 lanes the other one with a single lane 

In [3]:
# Traffic Network
trf_net = {"lengths_per_link": (20000, 10000), "lanes_per_link": (1, 2)}
traffic_network = TrafficNetwork(**trf_net)

## Traffic demand

In this case a randomized demand of half capacity is injected in each one of the lanes 

In [4]:
# Demand
demand1 = Demand((C / 2,), (12,))
demand2 = Demand((C / 2,), (12,))
trf_dmd = {"lks": (1,), "demands": (demand1, demand2)}

traffic_demand = TrafficDemand(**trf_dmd)

In [5]:
show(demand1.plot_demand_elements())

In [6]:
show(demand2.plot_demand_elements())

# Traffic Scenario 

The combination of a simulation and a demand generate a traffic scenario, the scenario can be used by the control in order to evolve the simulation 

In [7]:
case_study = Scenario(traffic_network, traffic_demand)

## Create control for scenario 

At this point vehicles are already initialized on each link so a control task is just required to evolve the vehicles on each link. 

* Leader of the formation will have a policy in terms of acceleration so that it will estimulate the change of lane 
* The control will initially evolve all vehicles as if the network was disconnected 

* Tasks to do: 
    1. Find the leader's acceleration based on the link_length and lane. It should be somehow automatically set
    2. Fix the acceleration, this can be based on a speed policy based in space this means. Leader takes a speed such that is a map from space v(x) leader query control and control is a function of the position. Do this by lane 
    3. You have two controllers. 
        1. is the general simulator control this one just observes the network and tells evolve and then checks if the vehicle should change lane or link 
        2. is a ITS control, this one checks at x distance. It queries all vehicles at vehicle t and checks those in radious<= treshold, then for those within the radious runs acceptance distance random, similar to T_ACCEPT, and 
           checks if position is greater than this create an speed drop (acc) if not do not modify. Finally all vehicles should decrease their speeds at specific points of space e.g 400m
           
 

A. Ladino 